In [181]:
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches

import pandas as pd
from svgpathtools import parse_path

In [184]:
# directory & file hierarchy
exp_path = 'museumdraw'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
##
sketch_dir = os.path.join(analysis_dir,'sketches')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)
## dir where we can keep all sketches not binned by category    

import sys
## add helpers to python path
if os.path.abspath(os.getcwd()) not in sys.path:
    sys.path.append(os.path.abspath(os.getcwd()))
    
import svg_distance_helpers as rsh
reload(rsh)

<module 'svg_distance_helpers' from 'svg_distance_helpers.pyc'>

In [185]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['cdm_run_v3']

In [186]:
imrecs = coll.find({'$and':[{'category':'square'},{'dataType':'finalImage'}]}).sort('time')
print imrecs.count()

## retrieve example square tracing from db
imrec = imrecs[90] 

2490


In [202]:
## get list of vertices and codes for tracing 
try:
    ## get the corresponding stroke records for this tracing
    stroke_recs = coll.find({'$and': [{'sessionId':imrec['sessionId']}, 
                                      {'dataType':'stroke'},
                                      {'trialNum': imrec['trialNum']}]}).sort('time')

    print 'Number of strokes in this tracing: {}'.format(stroke_recs.count())    
    
    ## check to make sure that there is at least one stroke! 
    if stroke_recs.count()>0:

        ## now make an svg list!
        svg_list = rsh.make_svg_list(stroke_recs)

        ## now get me some verts and codes!
        Verts, Codes = rsh.get_verts_and_codes(svg_list)
        
except Exception as e:
    print 'Oops, something went wrong! Here is the error:'  
    print e
    pass        

Number of strokes in this tracing: 1


In [203]:
## simplify tracing a bit if it is a jungle of verts/horiz commands
_Verts,_Codes = rsh.simplify_verts_and_codes(Verts,Codes)

In [205]:
## examine tracing
reallyRun = 0
if reallyRun:
    rsh.plot_shape(_Verts,_Codes)

In [207]:
## get vertices and codes for reference shape (in this example, a square)
Verts, Codes = rsh.get_nearest_reference_square_to_tracing(_Verts,_Codes)

In [214]:
## examine reference shape (in this example, a square)
reallyRun = 0
if reallyRun:
    rsh.plot_shape(Verts,Codes)

In [247]:
A = np.array((0,0))
B = np.array((10,10))
P = np.array((20,5)) ## first point

## get point C on A,B that minimizes distance from particular point on tracing to the line segment AB
C = rsh.get_closest_point_from_P_to_AB(A,B,P) 

t is 1.25
point lies outside A and B
